# 2. 检索器改进

本案例展示了在基础的RAG流程之上，尝试采用不同的检索器和重排序方法，观察问题回答的质量。

#### 0. 环境准备

In [1]:
!pip install -U langchain==0.2.11 openai==1.37.0 ragas==0.1.11 arxiv==2.1.3 pymupdf==1.24.9 chromadb==0.5.5 wandb==0.17.5 tiktoken==0.7.0 pypdf==4.3.1 sentence_transformers==2.7.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.2/154.2 kB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 11.8 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 6.8 MB/s eta 0:00:0000:0100:01mm
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

#### 1. 完成基础RAG流程代码

##### 导入pdf数据集

In [2]:
import os
from langchain.document_loaders import pdf, PyPDFLoader

pdf_folder_path = "/gemini/code/RiceBlastData"
base_docs = []
for file in os.listdir(pdf_folder_path):
  if file.endswith('.pdf'):
    pdf_path = os.path.join(pdf_folder_path, file)
    print(pdf_path)
    # 创建pdf的加载器实例，需要调用load方法以获取内容。
    loader = PyPDFLoader(pdf_path)
    base_docs.extend(loader.load())
len(base_docs)

/gemini/code/RiceBlastData/20-100 (1).pdf


Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet


/gemini/code/RiceBlastData/20-100.pdf


Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet
Advanced encoding /UniGB-UTF16-H not implemented yet


/gemini/code/RiceBlastData/202004-411-422.pdf
/gemini/code/RiceBlastData/2021-null.pdf
/gemini/code/RiceBlastData/2022-null.pdf


...

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)


/gemini/code/RiceBlastData/水稻稻瘟病-百度百科.pdf
/gemini/code/RiceBlastData/稻瘟病-百度百科.pdf


Ignoring wrong pointing object 6 0 (offset 0)


/gemini/code/RiceBlastData/稻瘟病菌.pdf
/gemini/code/RiceBlastData/稻瘟病菌群体遗传结构的研究进展.pdf


97

##### 定义ChatGLM3 LLM类

In [3]:
from langchain.llms.base import LLM
from typing import Any, List, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

class ChatGLM3_LLM(LLM):
    # 基于本地 InternLM 自定义 LLM 类
    tokenizer : AutoTokenizer = None
    model: AutoModelForCausalLM = None

    def __init__(self, model_path :str):
        # model_path: InternLM 模型路径
        # 从本地初始化模型
        super().__init__()
        print("正在从本地加载模型...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).to(torch.bfloat16).cuda()
        self.model = self.model.eval()
        print("完成本地模型的加载")

    def _call(self,
              prompt : str,
              stop: Optional[List[str]] = None,
              run_manager: Optional[CallbackManagerForLLMRun] = None,
              **kwargs: Any):
        # 重写调用函数
        response, history = self.model.chat(self.tokenizer, prompt , history=[])
        return response
        
    @property
    def _llm_type(self) -> str:
        return "ChatGLM3-6B"

/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### 创建提问模版

In [4]:
from langchain.prompts import ChatPromptTemplate

template = """请基于以下提供的上下文信息，回答如下问题。如果你认为根据提供的信息无法回答该问题，请回答'我不知道':

### 上下文信息
{context}

### 问题
问题: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

#### 2. 采用Parent Document Retriever（PDR）检索器

#### Parent Document Retriever

在检索增强生成（RAG）系统中，使用较小或较大的文本块进行信息检索是一个重大挑战。这种选择影响系统准确和上下文相关性。
- 较小的块大小意味着嵌入更精确，而较大的块大小意味着嵌入可能更通用（提供更多上下文），但可能会错过细节。较小的块在检索中提供高精度，将用户查询的特定细节与针对性、相关内容匹配。这最小化了噪声和无关信息，提高了响应的准确性。
- 较大的块提供了全面的上下文，这对于需要更广泛理解或本质上复杂的查询是必要的。这些块虽然精确度较低，但可以通过覆盖更广泛的信息范围提供更丰富的洞察。

RAG系统中的主要挑战是在对详细、特定信息的需求（较小块）与对更广泛上下文理解的要求（较大块）之间取得平衡。选择最优的块大小影响系统在生成响应的精确度和全面性方面的性能。
父文档检索器（PDR）旨在有效应对这一挑战。它们采用双层检索策略：
第一层：专注于检索更小、更精确的块，直接解决查询的细节。这确保响应是相关且切题的。
第二层：一旦确定了相关块，系统就检索这些块派生的较大父文档。这一步提供了额外的上下文和深度，通过支持详细信息的更广泛视角来丰富响应。

通过整合精确和上下文数据检索，PDR增强了RAG系统产生全面和准确响应的能力。它们使系统能够通过动态调整细节和广度之间的焦点来处理各种查询复杂性。

父文档检索器的好处：
- 提高精度和相关性：专注于较小块提高了对特定查询响应的相关性和精确度。
- 丰富的上下文理解：通过获取父文档，PDR提供了复杂查询所必需的更广泛上下文。
- 高效的信息检索：首先针对较小段，优化计算资源的使用。- 可扩展性和灵活性：可以调整检索的粒度，使PDR适应各种查询类型和用户需求。
- 提高大型语言模型（LLM）的性能：来自精确和广泛数据的丰富嵌入提高了LLM响应的准确性和上下文性。

PDR检索方法的基本流程概述如下：

1. 获取用户问题
2. 使用密集向量检索检索子文档
3. 根据它们的父文档合并子文档。如果它们有相同的父文档，则它们会被合并。
4. 用内存存储中的相应父文档替换子文档。
5. 使用父文档增强生成。

##### 动手练习1

- <1>处，创建一个RecursiveCharacterTextSplitter文本分割器parent_splitter，文本块大小设置为1500；
- <2>处，创建一个RecursiveCharacterTextSplitter文本分割器child_splitter，文本块大小设置为200；
- <3>处，创建名为`"split_parents"`向量存储库，指定embedding模型为前面部署已加载好的模型。

In [5]:
# 加载Embedding模型，并保存为向量数据库
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

EMBEDDING_PATH = "/gemini/pretrain/bge-m3"
embeddings = SentenceTransformerEmbeddings(model_name=EMBEDDING_PATH)

/root/miniconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [7]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

# <1>父文本块的大小为1500
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1500)
# <2>子文本块的大小为200
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

# 使用Chroma创建了向量存储库,命名为"split_parents",并使用之前定义的embeddings模型。
vectorstore = Chroma(collection_name="split_parents", embedding_function=embeddings)

store = InMemoryStore()

/root/miniconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


##### 动手练习2

<1>处创建ParentDocumentRetriever检索器，使用 vectorstore 来检索与查询最相关的文档向量，使用 store 来存储文档数据，使用 `child_splitter` 分割器来获取父文本块，使用 `parent_splitter` 来处理文档的子文本块。

<2>处，将base_docs文档列表添加入parent_document_retriever进行处理。

In [8]:
# <1>
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)
# <2>
parent_document_retriever.add_documents(base_docs)

#### 加载 ChatGLM3 模型

##### 动手练习3

- <1>处，设置ChatGLM3模型文件所在的路径；
- <2>处，创建ChatGLM3模型；

In [10]:
model_path = os.path.expandvars('$GEMINI_PRETRAIN2/chatglm3-6b')
primary_qa_llm = ChatGLM3_LLM(model_path=model_path)

正在从本地加载模型...


Loading checkpoint shards: 100%|██████████| 7/7 [01:34<00:00, 13.52s/it]


完成本地模型的加载


#### 创建提问链

In [11]:
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from operator import itemgetter

parent_document_retriever_qa_chain = (
    {"context": itemgetter("question") | parent_document_retriever,
     "question": itemgetter("question")
    }
    | RunnablePassthrough.assign(
        context=itemgetter("context")
      )
    | {
         "response": prompt | primary_qa_llm,
         "context": itemgetter("context"),
      }
  )

#### 调用提问链，获得结果

In [12]:
parent_document_retriever_qa_chain.invoke({"question" : "什么是稻瘟病?"})["response"]

'稻瘟病是一种由稻瘟病原菌引起的、发生在水稻上的疾病。它会导致水稻的叶片、茎、穗和节等部位受到病害的影响，从而影响水稻的产量和质量。稻瘟病在水稻整个生长周期中均可发生，对水稻造成毁灭性的影响。为了防治稻瘟病，人们通常采取选用抗病品种、培育优质秧苗、肥水管理技术措施、加强田間管理、落实防治措施和化学药剂控制等措施。'

#### 3. 集成检索 (Ensemble Retrieval)
接下来让我们看看集成检索！

基本思想如下：

1. 获取用户问题
2. 同时调用两个检索器：
- 使用BM25稀疏向量检索方法检索文档
- 使用密集向量检索方法检索文档
3. 根据它们的权重，使用互惠排名融合算法[Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf)收集并"融合"检索到的文档，形成一个单一的排名列表。
4. 使用这些文档来增强我们的生成。
确保您的权重列表——每个检索器的相对权重——加起来等于1！


In [13]:
!pip install rank_bm25

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


#### 动手练习4

- <1>处，创建一个 BM25Retriever 检索器实例bm25_retriever，使用 docs 作为其文档基础，检索输出文档数k设置为3；
- <2>处，创建一个 EnsembleRetriever 实例，集成两个检索器：bm25_retriever 和 chroma_retriever，设置 bm25_retriever 的权重是0.75，chroma_retriever 的权重是0.25。

In [14]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(base_docs)

bm25_retriever = BM25Retriever.from_documents(docs, k=3)

vectorstore = Chroma.from_documents(docs, embeddings)
chroma_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever],
    weights=[0.75, 0.25]
)

#### 创建提问链

In [15]:
ensemble_retriever_qa_chain = (
    {"context": itemgetter("question") | ensemble_retriever,
     "question": itemgetter("question")
    }
    | RunnablePassthrough.assign(
        context=itemgetter("context")
      )
    | {
         "response": prompt | primary_qa_llm,
         "context": itemgetter("context"),
      }
  )

#### 获取回答

In [16]:
ensemble_retriever_qa_chain.invoke({"question" : "什么是稻瘟病?"})["response"]

'稻瘟病是一种由真菌引起的植物病害，它严重危害水稻产量，主要通过稻草秸秆里的病菌孢子传播。稻瘟病会导致水稻叶片、茎秆、穗颈等部位出现病斑，病斑正反面均呈现褪绿色。如果不及时防治，稻瘟病会导致水稻减产40%至50%，甚至导致绝收。稻瘟病的防治主要包括选用抗病品种、培育优质秧苗、肥水管理技术措施、加强田间管理、化学药剂控制等。'